In [1]:
import numpy as np
import scipy.stats
import decimal                         # decide on given number how many decimal points to keep for values in array
from tqdm import tqdm_notebook as tqdm # check progress
from tabulate import tabulate          # nice print of table data

In [2]:
def euclidean_multiple_distance(p_array, p):
    return np.sqrt(np.square(p_array - p).sum(axis=1))

def gaussian_kernel(distance, param):
    return np.exp(-0.5 * np.square(distance/param['sigma'])) / np.power(param['sigma'] * np.sqrt(2*np.pi), param['dim'])

def normalize(data):
    # mitigate outliers, drop extreme 1% from each side
    data_slice = scipy.stats.trimboth(data, proportiontocut=0.01) 
    mean = np.mean(data_slice, axis=0)[np.newaxis]
    std = np.std(data_slice, axis=0).max()
    data_normed = (data - mean) / std
    return data_normed, mean, std

def denormalize(data, mean, std):
    return data * std + mean 

def meanShiftClustering(data, window, stop_criteria, param):
    """Find clusters using Mean-shift algorithm.
    
    Args: 
        data: numpy array
        window: float, how far to look for neighbor points
        stop_criteria: dict, 'epsilon' float distance boundary representing convergence, 'max_iter' int number of iterations making sure we are not stuck forever
        param: dict, parameters for kernel, now it holds 'sigma': float; and 'dim': int, dimensionality of input data points.
    
    Return:
        cluster_ids: numpy array, id of cluster to which data point has been assigned
        cluster_centers: numpy array, coordinates of estimated cluster centers
        converged: bool, whether process converged (hitted 'epsilon' criteria) or not (hitted 'max_iter' criteria)
    """
    def mean_shift_iteration(data, point, window, distance_multiple_func, kernel, param):
        # find neighbor points
        distances = distance_multiple_func(data, point)
        neighbors_idx = np.where(distances < window)[0]
        
        # select neighbors
        neighbors = data[neighbors_idx]
        neighbors_distances = distances[neighbors_idx]
        
        # compute summation coefficients
        weights = kernel(neighbors_distances, param)
        
        # get centroid as a weighted average
        centroid = (weights[:, np.newaxis] * neighbors).sum(axis=0) / weights.sum()
        return centroid

    distance_multiple_func = euclidean_multiple_distance
    kernel = gaussian_kernel

    max_iter = stop_criteria['max_iter']
    epsilon = stop_criteria['epsilon']

    converged = False
    data = data.copy()
    data, mean, std = normalize(data)  # so that we can keep hyperparams more-less in the same range
    param['dim'] = data.shape[1]
    
    for i in tqdm(range(max_iter)):
        # apply mean shift to all points at the same time (shift all points to mean)
        data_new = np.stack([mean_shift_iteration(data, point, window, distance_multiple_func, kernel, param) for point in data])
        
        # check what the largest shift was
        max_shift = np.max(distance_multiple_func(data_new, data))
        data = data_new
    
        # if largest shift was small enough - converged - terminate algorithm
        if max_shift < epsilon:
            converged = True
            break
    
    # round to the one digit more coerce level than epsilon has digits after comma
    # that is to make sure all points of a cluster have exactly the same value
    rounding_digits = -1 - decimal.Decimal(str(epsilon)).as_tuple().exponent
    data = np.round(data, rounding_digits)
    
    # return to the original scale
    data = denormalize(data, mean, std)
    
    # find cluster center positions
    cluster_centers = np.array(list(set(tuple(p) for p in data)))
    
    # assign points to clusters 
    cluster_ids = np.array([i for point in data for i, centroid in enumerate(cluster_centers) if np.all(point == centroid)])
    
    return cluster_ids, cluster_centers, converged

# Test

Due to data scaling, meaningful results can be obtained for multiple datasets with hyperparams as is below. 

Except for window (size), which should be fine-tuned for each dataset separately, because clusters have different relative sizes.  

In [3]:
stop_criteria = {'max_iter': 100, 'epsilon': 1e-8}
param = {'sigma': 1}

## G set

In [4]:
gt_cluster_centers = np.loadtxt('data/g2-gt-txt/g2-8-60-gt.txt')
data = np.loadtxt('data/g2-txt/g2-8-60.txt')
window = 1.25 * np.sqrt(data.shape[1])  # later multiplicator is to account for distance scaling with rise of dimensionality

print("Number of ground truth clusters:", gt_cluster_centers.shape[0])

Number of ground truth clusters: 2


In [5]:
cluster_ids, cluster_centers, converged = meanShiftClustering(data, window, stop_criteria, param)
print("Converged:",  converged)
print("Number of clusters:", cluster_centers.shape[0])


Converged: True
Number of clusters: 2


In [6]:
rounding_digits = -decimal.Decimal(gt_cluster_centers.reshape(-1)[0]).as_tuple().exponent
cluster_centers_rounded = np.round(cluster_centers, rounding_digits)

# find closest cluster to the ground truth from calculated
matches = np.array([cluster_centers_rounded[np.argmin(euclidean_multiple_distance(cluster_centers_rounded, gt))] for gt in gt_cluster_centers])

# calculate differences in positions between linked calculated clusters and ground truth clusters 
diffs = matches - gt_cluster_centers

# xi - coordinates of linked calculated clusters, dxi - differences with ground truth 
print(tabulate(np.concatenate((matches, diffs), axis=1).tolist(), headers=[*[f'x{i}' for i in range(data.shape[1])], *[f'dx{i}' for i in range(data.shape[1])]]))

  x0    x1    x2    x3    x4    x5    x6    x7    dx0    dx1    dx2    dx3    dx4    dx5    dx6    dx7
----  ----  ----  ----  ----  ----  ----  ----  -----  -----  -----  -----  -----  -----  -----  -----
 504   500   503   500   504   508   504   504      4      0      3      0      4      8      4      4
 597   597   600   597   596   596   596   593     -3     -3      0     -3     -4     -4     -4     -7


In [7]:
idx, count = np.unique(cluster_ids, return_counts=True)
print(tabulate(np.stack((idx, count), axis=1), headers=['cluster_id', 'count']))

  cluster_id    count
------------  -------
           0     1021
           1     1027


## S set

In [8]:
gt_cluster_centers = np.loadtxt('data/s-originals/s1-cb.txt')
data = np.loadtxt('data/s/s1.txt')
window = 0.275 * np.sqrt(data.shape[1])

print("Number of ground truth clusters:", gt_cluster_centers.shape[0])

Number of ground truth clusters: 15


In [9]:
cluster_ids, cluster_centers, converged = meanShiftClustering(data, window, stop_criteria, param)
print("Converged:",  converged)
print("Number of clusters:", cluster_centers.shape[0])


Converged: True
Number of clusters: 16


Note: we have one excessive cluster, keep that in mind.

In [10]:
rounding_digits = -decimal.Decimal(gt_cluster_centers.reshape(-1)[0]).as_tuple().exponent
cluster_centers_rounded = np.round(cluster_centers, rounding_digits)
matches = np.array([cluster_centers_rounded[np.argmin(euclidean_multiple_distance(cluster_centers_rounded, gt))] for gt in gt_cluster_centers])
diffs = matches - gt_cluster_centers
print(tabulate(np.concatenate((matches, diffs), axis=1).tolist(), headers=[*[f'x{i}' for i in range(data.shape[1])], *[f'dx{i}' for i in range(data.shape[1])]]))

    x0      x1    dx0    dx1
------  ------  -----  -----
606294  571377   1966  -3002
802684  319603    776   1221
416341  787615    -42   1411
823377  729719    606  -2315
851692  158250    699    377
335509  561760  -3077  -1777
167922  347818  -1352   -756
617676  400922  -1583   3251
246121  846950   5050   2526
322476  162581    675  -2738
141611  557544   2118    192
506510  175900  -2275   1100
398885  405346    -49   1204
858793  547291  -2065   1232
672172  861813  -2193   1349


In [11]:
idx, count = np.unique(cluster_ids, return_counts=True)
print(tabulate(np.stack((idx, count), axis=1), headers=['cluster_id', 'count']))

  cluster_id    count
------------  -------
           0      351
           1      341
           2      345
           3      339
           4      311
           5      314
           6        1
           7      297
           8      318
           9      329
          10      340
          11      334
          12      353
          13      350
          14      327
          15      350


So cluster 6 is a single isolated point, it (and overly small clusters in general) can be reassigned to the closest clusters, discarded altogether, etc during post processing stage. 

I will leave it be for now. 